In [1]:
from models.graph import Graph, load_graph
from models.interaction_network import InteractionNetwork
from utils import *
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import pyopencl as cl
import time
import h5py

prep, pt_cut = 'LP', 5
graph_dir = "../IN_{0}_{1}/".format(prep, pt_cut)
model_path = './trained_models/LP_2_1500_wide_noPhi_epoch300.pt'
data_path = '../data'
#kernelSource = open('kernel_code.cl').read()
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
program = cl.Program(ctx, kernelSource).build()
mf = cl.mem_flags

graphs = get_graphs(graph_dir)
test_size = 100
objects, sender_relations, receiver_relations, relation_info, y = get_inputs(graphs[:test_size])

weights = torch.load(model_path)

In [2]:
object_dim, relation_dim, effect_dim = 3, 1, 1
interaction_network = InteractionNetwork(object_dim, relation_dim, effect_dim)

interaction_network.load_state_dict(weights)
interaction_network.eval()
start = time.time()
pred = interaction_network(objects, sender_relations, receiver_relations, relation_info)
end = time.time()
pytorch_pred = torch.cat(pred, dim=0)
target = torch.cat(y, dim=0)
pytorch_pred.shape, target.shape, end - start

(torch.Size([7803, 1]), torch.Size([7803, 1]), 0.22286319732666016)

hf = h5py.File('{}/test.hdf5'.format(data_path), 'w')
obj_ms = []
sr_ms = []
rr_ms = []
ri_ms = []
obj_ns = []
sr_ns = []
rr_ns = []
ri_ns = []

for i in range(test_size):
    obj = objects[i].numpy()#.tolist()
    sr = sender_relations[i].numpy()#.tolist()
    rr = receiver_relations[i].numpy()#.tolist()
    ri = relation_info[i].numpy()#.tolist()
    hf.create_dataset('obj_{}'.format(i), data=obj.tolist())
    hf.create_dataset('sr_{}'.format(i), data=sr.tolist())
    hf.create_dataset('rr_{}'.format(i), data=rr.tolist())
    hf.create_dataset('ri_{}'.format(i), data=ri.tolist())
    
    obj_ms.append(obj.shape[0])
    sr_ms.append(sr.shape[0])
    rr_ms.append(rr.shape[0])
    ri_ms.append(ri.shape[0])
    
    obj_ns.append(obj.shape[1])
    sr_ns.append(sr.shape[1])
    rr_ns.append(rr.shape[1])
    ri_ns.append(ri.shape[1])
    
hf.create_dataset('obj_shape_0_i', data=obj_ms)
hf.create_dataset('sr_shape_0_i', data=sr_ms)
hf.create_dataset('rr_shape_0_i', data=rr_ms)
hf.create_dataset('ri_shape_0_i', data=ri_ms)

hf.create_dataset('obj_shape_1_i', data=obj_ns)
hf.create_dataset('sr_shape_1_i', data=sr_ns)
hf.create_dataset('rr_shape_1_i', data=rr_ns)
hf.create_dataset('ri_shape_1_i', data=ri_ns)
hf.close()
hf = h5py.File('{}/model_weights.hdf5'.format(data_path), 'w')
for k in weights.keys():
    weight = weights[k].t()
    hf.create_dataset(k, data=weight.numpy().tolist())
hf.close()

In [2]:
print(len(objects), len(sender_relations), len(receiver_relations), len(relation_info))
for i in range(3):
    obj = objects[i].numpy()
    sr = sender_relations[i].numpy()
    rr = receiver_relations[i].numpy()
    ri = relation_info[i].numpy()
    
    print(obj.shape, sr.shape, rr.shape, ri.shape)

10 10 10 10
(34, 3) (34, 38) (34, 38) (1, 38)
(55, 3) (55, 41) (55, 41) (1, 41)
(51, 3) (51, 59) (51, 59) (1, 59)


In [3]:
def cl_transpose(a):
    n, m = a.shape[0], a.shape[1]
    a = a.flatten().astype(np.float32)
    a_t = np.zeros((m, n), dtype=np.float32)
    a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
    a_t_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a_t.nbytes)
    program.transpose(queue, 
                      (n, m), 
                      None, 
                      a_t_buf, 
                      a_buf, 
                      np.uint8(n), 
                      np.uint8(m))
    cl.enqueue_copy(queue, a_t, a_t_buf)
    return a_t

def cl_matmul(a, b):
    n, m, p = a.shape[0], a.shape[1], b.shape[1]
    c = np.zeros((n*p), dtype=np.float32)
    a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
    b_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
    c_buf = cl.Buffer(ctx, mf.WRITE_ONLY, c.nbytes)
    program.matMul(queue,
                   c.shape,
                   None,
                   a_buf,
                   b_buf,
                   c_buf,
                   np.uint8(n),
                   np.uint8(m),
                   np.uint8(p))
    d = np.zeros((n,p), dtype=np.float32)
    cl.enqueue_copy(queue, d, c_buf)
    return d

def relu(inp):
    out = np.zeros(inp.shape, dtype=np.float32)
    inp_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=inp)
    out_buf = cl.Buffer(ctx, mf.WRITE_ONLY, out.nbytes)
    program.relu(queue,
                 inp.shape,
                 None,
                 inp_buf,
                 out_buf,
                 np.uint8(inp.shape[1]))
    cl.enqueue_copy(queue, out, out_buf)
    return out

def sigmoid(inp):
    out = np.zeros(inp.shape, dtype=np.float32)
    inp_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=inp)
    out_buf = cl.Buffer(ctx, mf.WRITE_ONLY, out.nbytes)
    program.sigmoid(queue,
                 inp.shape,
                 None,
                 inp_buf,
                 out_buf,
                 np.uint8(inp.shape[1]))
    cl.enqueue_copy(queue, out, out_buf)
    return out

def add_bias(inp, bias):
    out = np.zeros(inp.shape, dtype=np.float32)
    inp_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=inp)
    bias_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=bias)
    out_buf = cl.Buffer(ctx, mf.WRITE_ONLY, out.nbytes)
    program.add_bias(queue,
                 inp.shape,
                 None,
                 inp_buf,
                 bias_buf,
                 out_buf,
                 np.uint8(inp.shape[1]))
    cl.enqueue_copy(queue, out, out_buf)
    return out
'''
def linear(inp, weight, bias):
    weight_t = np.zeros((weight.shape[1], weight.shape[0]), dtype=np.float32)
    out_mm = np.zeros((inp.shape[0], weight.shape[0]), dtype=np.float32)
    out = np.zeros((inp.shape[0], weight.shape[0]), dtype=np.float32)
    
    inp_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=inp)
    weight_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=weight)
    weight_t_buf = cl.Buffer(ctx, mf.COPY_HOST_PTR, hostbuf=weight_t)
    bias_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=bias)
    out_mm_buf = cl.Buffer(ctx, mf.COPY_HOST_PTR, hostbuf=out_mm)
    
    out_buf = cl.Buffer(ctx, mf.WRITE_ONLY, out.nbytes)
    
    c1, c2 = weight.shape
    c3 = inp.shape[0] * weight.shape[1]
    inp_n = inp.shape[0]
    
    program.linear(queue,
                 (c1, c2),
                 (c3, inp_n),
                 inp_buf,
                 weight_buf,
                 weight_t_buf,
                 out_mm_buf,
                 bias_buf,
                 out_buf,
                 np.uint8(c1),
                 np.uint8(c2),
                 np.uint8(inp_n))
    cl.enqueue_copy(queue, out, out_buf)
    return out
'''
def linear(inp, weight, bias):
    w_t = cl_transpose(weight) # w1, w0
    inp = cl_matmul(inp, w_t) # inp0, w0
    out = add_bias(inp, bias) # inp0, w0
    return out

def relational_model(inp, weights):
    inp = cl_transpose(inp)
    
    RM_L0 = linear(inp, 
                   weights['relational_model.layers.0.weight'].numpy(), 
                   weights['relational_model.layers.0.bias'].numpy())
    RM_L0 = relu(RM_L0)

    RM_L2 = linear(RM_L0, 
                   weights['relational_model.layers.2.weight'].numpy(), 
                   weights['relational_model.layers.2.bias'].numpy())
    RM_L2 = relu(RM_L2)
    RM_L4 = linear(RM_L2, 
                   weights['relational_model.layers.4.weight'].numpy(), 
                   weights['relational_model.layers.4.bias'].numpy())
    RM_L4 = relu(RM_L4)

    RM_L6 = linear(RM_L4, 
                   weights['relational_model.layers.6.weight'].numpy(), 
                   weights['relational_model.layers.6.bias'].numpy())
    RM_L6 = sigmoid(RM_L6)
    return RM_L6

def object_model(inp, weights):
    inp = cl_transpose(inp)
    
    OM_L0 = linear(inp, 
                   weights['object_model.layers.0.weight'].numpy(), 
                   weights['object_model.layers.0.bias'].numpy())
    OM_L0 = relu(OM_L0)

    OM_L2 = linear(OM_L0, 
                   weights['object_model.layers.2.weight'].numpy(), 
                   weights['object_model.layers.2.bias'].numpy())
    OM_L2 = relu(OM_L2)

    OM_L4 = linear(OM_L2, 
                   weights['object_model.layers.4.weight'].numpy(), 
                   weights['object_model.layers.4.bias'].numpy())
    
    return OM_L4

def interaction_cat(term_w, term_h, sender, receiver, ri):
    out = np.zeros((term_w, term_h), dtype=np.float32)

    sender_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = sender)
    receiver_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = receiver)
    ri_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = ri)
    out_buf = cl.Buffer(ctx, mf.WRITE_ONLY, out.nbytes)
    program.interaction_cat(queue,
                 out.shape,
                 None,
                 sender_buf,
                 receiver_buf,
                 ri_buf,
                 out_buf,
                 np.uint8(out.shape[1]))
    cl.enqueue_copy(queue, out, out_buf)
    return out

def aggregate_cat(obj_t, effect_receiver):
    term_w = obj_t.shape[0] + effect_receiver.shape[0]
    term_h = obj_t.shape[1]
    out = np.zeros((term_w, term_h), dtype=np.float32)

    obj_t_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = obj_t)
    effect_receiver_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf = effect_receiver)
    out_buf = cl.Buffer(ctx, mf.WRITE_ONLY, out.nbytes)
    program.aggregate_cat(queue,
                 out.shape,
                 None,
                 obj_t_buf,
                 effect_receiver_buf,
                 out_buf,
                 np.uint8(out.shape[1]))
    cl.enqueue_copy(queue, out, out_buf)
    return out

In [4]:
# this method needs to be a kernel
def interaction_network_forward(objects,
                             sender_relations,
                             receiver_relations,
                             relation_info):
    predicted = []
    N = len(objects)
    for i in range(N):
        obj = objects[i].numpy()
        sr = sender_relations[i].numpy()
        rr = receiver_relations[i].numpy()
        ri = relation_info[i].numpy()

        obj_h, sr_h, ri_w = obj.shape[1], sr.shape[1], ri.shape[0]
        obj_t = cl_transpose(obj)

        sender = cl_matmul(obj_t, sr)
        receiver = cl_matmul(obj_t, rr)
        
        term_w, term_h = obj_h + obj_h + ri_w, sr_h
        
        interaction_term = interaction_cat(term_w, term_h, sender, receiver, ri)
        
        
        
        effect = relational_model(interaction_term, weights)
        effect_receiver = cl_transpose(cl_matmul(rr, effect))
        
        aggregate = aggregate_cat(obj_t, effect_receiver)
        
        inf = object_model(aggregate, weights)
        predict = cl_transpose(inf)

        sender = cl_matmul(predict, sr)
        receiver = cl_matmul(predict, rr)
        
        predict_shape_0 = 3
        term_w, term_h = predict_shape_0 + predict_shape_0 + ri_w, sr_h
        interaction_term = interaction_cat(term_w, term_h, sender, receiver, ri)
        
        predict = relational_model(interaction_term, weights)
        predicted.append(predict)
    return predicted

In [5]:
start = time.time()
pr = interaction_network_forward(objects,
                             sender_relations,
                             receiver_relations,
                             relation_info)
end = time.time()
end - start

3 38 1
3 41 1
3 59 1
3 56 1
3 26 1
3 75 1
3 174 1
3 202 1
3 104 1
3 70 1
3 64 1
3 31 1
3 75 1
3 67 1
3 84 1
3 146 1
3 54 1
3 47 1
3 120 1
3 26 1
3 68 1
3 122 1
3 34 1
3 37 1
3 45 1
3 21 1
3 65 1
3 175 1
3 112 1
3 56 1
3 29 1
3 65 1
3 20 1
3 30 1
3 28 1
3 164 1
3 88 1
3 46 1
3 38 1
3 42 1
3 194 1
3 55 1
3 68 1
3 58 1
3 42 1
3 66 1
3 51 1
3 117 1
3 65 1
3 70 1
3 73 1
3 214 1
3 35 1
3 45 1
3 33 1
3 40 1
3 50 1
3 130 1
3 101 1
3 149 1
3 38 1
3 18 1
3 35 1
3 36 1
3 72 1
3 151 1
3 44 1
3 105 1
3 35 1
3 98 1
3 38 1
3 47 1
3 106 1
3 93 1
3 33 1
3 25 1
3 46 1
3 33 1
3 46 1
3 52 1
3 29 1
3 44 1
3 188 1
3 672 1
3 56 1
3 51 1
3 25 1
3 67 1
3 125 1
3 193 1
3 102 1
3 62 1
3 96 1
3 141 1
3 89 1
3 57 1
3 30 1
3 27 1
3 61 1
3 37 1


2.600367784500122

In [9]:
pr

[array([[9.9999702e-01],
        [9.9194771e-01],
        [3.4225833e-01],
        [5.5628043e-01],
        [3.1034541e-01],
        [9.2776471e-01],
        [2.8138110e-01],
        [6.4978558e-01],
        [9.9995506e-01],
        [1.0000000e+00],
        [9.9998844e-01],
        [9.9999964e-01],
        [8.2071537e-01],
        [1.5129023e-02],
        [5.1678135e-04],
        [1.8031066e-02],
        [5.3423470e-01],
        [3.3079788e-01],
        [1.9363865e-06],
        [5.0826244e-02],
        [6.2802190e-01],
        [9.9995685e-01],
        [1.0000000e+00],
        [9.9997950e-01],
        [9.9999452e-01],
        [8.5748506e-01],
        [1.7392228e-03],
        [6.8777194e-06],
        [1.0000000e+00],
        [4.2371401e-05],
        [8.4828436e-01],
        [1.1738180e-01],
        [3.3379226e-08],
        [1.4355879e-01],
        [7.9811513e-01],
        [9.9961323e-01],
        [1.0000000e+00],
        [9.9999511e-01]], dtype=float32),
 array([[1.        ],
        [0.

In [ ]:
dynamic arrays? >> documentation for opencl?
5 - 10 min pres for computing meeting...
check in with isobel, savannah, and gage about gnn project update

In [ ]:
a exam?
intern for next summer? jpl / apl?
>> jobs? internship all but guarantees jobs.
>> 
nstrf?
7800 >> 9500